In [10]:
# %pip install openai==1.42.0
# %pip install langchain==0.2.14
# %pip install langchain_community==0.2.12
# %pip install transformers==4.44.2
# %pip install langchain_ollama==0.1.2

In [41]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from openai import OpenAI

In [14]:
# model = "gemma:2b"
model = "qwen2:1.5b"

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

NameError: name 'OpenAI' is not defined

In [43]:
def get_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content


In [44]:
get_completion('Hello. What up in the sky')

"It sounds like you're looking up at the sky and wondering what's going on!\n\nThere are many things that can be seen in the sky, depending on your location and time of day. Here are a few possibilities:\n\n* Clouds: You might see fluffy white clouds drifting lazily across the sky.\n* Planes: Commercial airliners or private planes might be flying overhead, leaving behind contrails (trails of condensed water vapor).\n* Birds: Birds like geese, ducks, or even eagles might be soaring through the skies.\n* Weather phenomena: Depending on your location and time of year, you might see things like sunsets, rainbows, or even thunderstorms brewing in the distance.\n* Stars and planets: If it's nighttime, you might see stars twinkling above, or even a planet like the Moon or Venus shining brightly.\n\nWhat do you think you're seeing up there?"

In [32]:
get_completion("What is 1+1?")

'The sum of 1 and 1 equals 2.'

In [59]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
style = """American English \
in a calm and respectful tone
"""
prompt = f"""Translate the text that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [60]:
get_completion(prompt)

'Sure, here is the translated text in a style that is American English in a calm and respectful tone:\n\n"Oh no! My blender lid flew off and splattered my kitchen walls with smoothie. Not only is the warranty not covering the cleaning costs, but I\'m also out of luck on a replacement. I need your help right away, mate!"'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [4]:
from langchain.llms import Ollama
from langchain_ollama import ChatOllama

In [5]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0

# Initialize the Ollama model
ollama_model = Ollama(model=model, temperature=0.0)
print(ollama_model)

chat_ollama_model = ChatOllama(model=model,temperature=0.0,)
print(chat_ollama_model)

Ollama
Params: {'model': 'qwen2:1.5b', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': 0.0, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
model='qwen2:1.5b' temperature=0.0 _client=<ollama._client.Client object at 0x10d88ba90> _async_client=<ollama._client.AsyncClient object at 0x10d92e500>


In [38]:
response = ollama_model("Tell me a joke.")
print(response)
print('---------')
response = chat_ollama_model.invoke("Tell me a joke.")
print(response.content)

Sure, here's one:

Why did the tomato turn red?

Because it saw the salad dressing!
---------
Sure, here's one:

Why did the tomato turn red?

Because it saw the salad dressing!


In [9]:
template_string = """Translate the text that is delimited by triple backticks \
into a style that is {style}. text: ```{text}```
"""


In [10]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(template_string)
# OR
_prompt = PromptTemplate(input_variables=["style","text"], template=template_string)
print(prompt_template)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'


# Why use prompt templates
- prompts can be long and detailed.
- reuse good prompt when you can.
- Langchain also provides prompts for common operations.
    - Langchain library functions parse the LLM's output assuming that it will use certain keywords.
    - eg: Thought, Action, Observation

In [11]:
customer_style = "American English in a calm and respectful tone"
customer_email = """Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help right now, matey!
"""

In [12]:
customer_messages = prompt_template.format(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(customer_messages)

<class 'str'>
Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [13]:
# Call the LLM to translate to the style of the customer message
customer_response = ollama_model(customer_messages)
print(customer_response)

/Users/rabinphaiju/Projects/GenAI/Langchain/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Sure, here is the translated text in a style that is American English in a calm and respectful tone:

"Oh no! I'm so frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! Not only that, but the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, mate!"


In [14]:
service_reply = """Hey there customer, the warranty does not cover \
cleaning expenses for your kitchen because it's your fault that \
you misused your blender by forgetting to put the lid on before \
starting the blender. Tough luck! See ya!
"""
service_style_pirate = """a polite tone that speaks in English Pirate"""

In [15]:
service_messages = prompt_template.format(
    style=service_style_pirate,
    text=service_reply)

print(service_messages)

service_response = ollama_model(service_messages)
print(service_response)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```

Ahoy there, mateys! Gather 'round, for I be speakin' a word of wisdom. The warranty be no kinder on a sailor who be forgettin' to put the lid on his blender, for that be a right royal case of a bad mistake. Be prepared to face the consequences, or be prepared to be shipwrecked by the wrath of the kitchen gods. So, be careful, or you'll be regrettin' this, me hearties!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [16]:
json_format = """{
  "gift": false,
  "delivery_days": 0,
  "price_value": ""
}"""

In [17]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's anniversary present. \
I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as clean JSON string with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [18]:
prompt_template = PromptTemplate.from_template(review_template)
print(prompt_template)
messages = prompt_template.format(text=customer_review)
response = ollama_model(messages)

input_variables=['text'] template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as clean JSON string with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'


In [19]:
print(response)

{
  "gift": True,
  "delivery_days": 2,
  "price_value": "slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


### Parse the LLM output string into a Python dictionary

In [22]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [30]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? \
                             Answer true if yes, false if not or unknown."
                             ,type="boolean")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product\
                                      to arrive? If this information is not found, output -1.",
                                      type="integer")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or \
                                    price, and output them as a comma separated Python list.",
                                    type="list")

response_schemas = [gift_schema, delivery_days_schema,price_value_schema]

In [31]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": boolean  // Was the item purchased as a gift for someone else?                              Answer true if yes, false if not or unknown.
	"delivery_days": integer  // How many days did it take for the product                                      to arrive? If this information is not found, output -1.
	"price_value": list  // Extract any sentences about the value or                                     price, and output them as a comma separated Python list.
}
```


In [37]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = PromptTemplate.from_template(template=review_template_2)

messages = prompt.format(text=customer_review, format_instructions=format_instructions)

In [38]:
print(messages)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "

In [39]:
response = ollama_model(messages)
print(response)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["The item's price is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [41]:
try:
    parsed_output = output_parser.parse(response)
    print(parsed_output)
    print("delivery_days:",parsed_output.get('delivery_days'))
except Exception as e:
    print(f"OutputParserException: {e}")


{'gift': True, 'delivery_days': 2, 'price_value': ["The item's price is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}
delivery_days: 2


## Prompt Template + LLM + Output Parsers

In [9]:
from langchain_ollama import ChatOllama
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [10]:
class CommaSeparatedListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        return text.split(",")

In [11]:
template = "Your are a helpful assistant. When the use given any input, you should generate 5 wrods synonyms of the input."
human_template = "{text}"

chatPrompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])
print(chatPrompt)

input_variables=['text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Your are a helpful assistant. When the use given any input, you should generate 5 wrods synonyms of the input.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


In [15]:
chat_llm =  ChatOllama(model=model,temperature=0.0)
chain = chatPrompt | chat_llm | CommaSeparatedListOutputParser()

In [19]:
chain.invoke({"text":"love"})

['affection', ' adoration', ' affectionate', ' fondness', ' passion']